In [1]:
from datastore import DataStore as ds


# ds.config.enable_debug()


nat = ds.from_file('../tests/dataset/users.csv')
nat.sql("select 1").execute()

nat = nat.filter(nat.age < 35)

nat['age_minus_10'] = nat['age'] - 10

nat = nat.add_prefix('col_').filter(nat.col_age > 25).filter(nat.col_country == 'USA')

nat['doubled'] = nat['col_age'] * 2

nat2 = nat.sql("doubled > 35")
nat2 = nat2.filter(nat2.doubled > 25)
nat2["float_age"] = nat2.doubled.cast("Float64")
print('Executing...')
# nat2.explain()

nat2["col_age"]
# df = nat2.to_df()
# print(df)

Executing...


0    28
1    31
2    29
Name: col_age, dtype: int64

In [2]:
nat2["col_name"].str.upper()
nat2.to_df()["col_name"].str.upper()
# nat2["col_name"].upper()


0     ALICE SMITH
1    DIANA PRINCE
2       GRACE LEE
Name: col_name, dtype: object

In [19]:
# Aggregate functions
from datastore.expressions import col
agg_ds = ds.from_file('../tests/dataset/orders.csv')
print(agg_ds.head())

agg_ds.groupby('product_id').agg(
    total_quantity=col('quantity').sum(),
    total_revenue=agg_ds.amount.sum(),
    stddev = col('amount').stddev_samp(),
    stddev2 = agg_ds.amount.stddev_samp()
).to_df()



   order_id  user_id  product_id  quantity order_date  amount
0      1001        1         101         2 2024-01-10   59.98
1      1002        2         102         1 2024-01-15  129.99
2      1003        1         103         3 2024-01-20   89.97
3      1004        3         101         1 2024-01-25   29.99
4      1005        4         104         2 2024-02-01  199.98


,product_id,total_quantity,total_revenue,stddev,stddev2
0,104,5,499.95,57.729253,57.729253
1,105,4,319.96,113.122943,113.122943
2,101,8,239.92,42.412265,42.412265
3,103,6,179.94,29.990000,29.990000
4,102,4,519.96,75.049761,75.049761
